In [216]:
import numpy as np

import pandas as pd
# Available in the github repo : examples/data/BTC_USD-Hourly.csv
url = "https://raw.githubusercontent.com/ClementPerroud/Gym-Trading-Env/main/examples/data/BTC_USD-Hourly.csv"
df = pd.read_csv(url, parse_dates=["date"], index_col= "date")
df.sort_index(inplace= True)
df.dropna(inplace= True)
df.drop_duplicates(inplace=True)

In [217]:
# df is a DataFrame with columns : "open", "high", "low", "close", "Volume USD"

# Create the feature : ( close[t] - close[t-1] )/ close[t-1]
df["feature_close"] = df["close"].pct_change()

# Create the feature : open[t] / close[t]
df["feature_open"] = df["open"]/df["close"]

# Create the feature : high[t] / close[t]
df["feature_high"] = df["high"]/df["close"]

# Create the feature : low[t] / close[t]
df["feature_low"] = df["low"]/df["close"]

 # Create the feature : volume[t] / max(*volume[t-7*24:t+1])
df["feature_volume"] = df["Volume USD"] / df["Volume USD"].rolling(7*24).max()

df.dropna(inplace= True) # Clean again !
# Eatch step, the environment will return 5 inputs  : "feature_close", "feature_open", "feature_high", "feature_low", "feature_volume"

In [218]:
import gymnasium as gym
import environments

#from environments import TradingEnv

In [219]:
import gym_trading_env

In [220]:
#import gymnasium as gym
#import gym_trading_env
#env = gym.make('MultiDatasetTradingEnv',
#    dataset_dir = 'preprocessed_data/*.pkl',
#    positions=[-1, 0, 1],  # -1 (=SHORT), 0(=OUT), +1 (=LONG)
#)

In [221]:
# Run 10 episodes
#for _ in range(10):
  # At every episode, the env will pick a new dataset.
#  done, truncated = False, False
#  observation, info = env.reset()
#  while not done and not truncated:
#      position_index = env.action_space.sample() # Pick random position index
#      observation, reward, done, truncated, info = env.step(position_index)

In [222]:
from environments import TradingEnv

# Create the environment directly
env = TradingEnv(
    name="BTCUSD",
    df=df,  # Your dataset with your custom features
    positions=[-0.8, 0.0, 0.8],  # -1 (=SHORT), 0(=OUT), +1 (=LONG)
    trading_fees=0.01/100,  # 0.01% per stock buy/sell (Binance fees)
    borrow_interest_rate=0.0003/100,  # 0.0003% per timestep (one timestep = 1h here)
)

In [223]:
from stable_baselines3 import PPO
from stable_baselines3 import SAC
from stable_baselines3.common.vec_env import VecEnv

In [224]:
policy = "MlpPolicy"

In [225]:
model = PPO(policy, env, verbose=1)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [226]:
# Experiment with different hyperparameters based on your problem and environment
model = PPO(
    "MlpPolicy",
    env,
    verbose=1,
    learning_rate=0.001,  # Adjust as needed
    batch_size=32,  # Adjust based on memory and learning speed
    gamma=0.99,  # Discount factor
)


Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [227]:
model.learn(total_timesteps=1000000)

-----------------------------
| time/              |      |
|    fps             | 1117 |
|    iterations      | 1    |
|    time_elapsed    | 1    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 737         |
|    iterations           | 2           |
|    time_elapsed         | 5           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.012114517 |
|    clip_fraction        | 0.173       |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.09       |
|    explained_variance   | -26.1       |
|    learning_rate        | 0.001       |
|    loss                 | -0.0417     |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.00849    |
|    value_loss           | 0.00257     |
-----------------------------------------
----------------------------------

-----------------------------------------
| time/                   |             |
|    fps                  | 553         |
|    iterations           | 13          |
|    time_elapsed         | 48          |
|    total_timesteps      | 26624       |
| train/                  |             |
|    approx_kl            | 0.004885153 |
|    clip_fraction        | 0.0296      |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.02       |
|    explained_variance   | -0.0278     |
|    learning_rate        | 0.001       |
|    loss                 | -0.0306     |
|    n_updates            | 120         |
|    policy_gradient_loss | -0.00154    |
|    value_loss           | 0.00068     |
-----------------------------------------
------------------------------------------
| time/                   |              |
|    fps                  | 560          |
|    iterations           | 14           |
|    time_elapsed         | 51           |
|    total_timesteps      | 2

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 3.31e+04    |
|    ep_rew_mean          | -2.89       |
| time/                   |             |
|    fps                  | 589         |
|    iterations           | 23          |
|    time_elapsed         | 79          |
|    total_timesteps      | 47104       |
| train/                  |             |
|    approx_kl            | 0.004374957 |
|    clip_fraction        | 0.0146      |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.04       |
|    explained_variance   | 0.00387     |
|    learning_rate        | 0.001       |
|    loss                 | 0.0103      |
|    n_updates            | 220         |
|    policy_gradient_loss | 0.000204    |
|    value_loss           | 0.0003      |
-----------------------------------------
----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 3.31e+04

Market Return : 423.10%   |   Portfolio Return : -78.93%   |   
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 3.31e+04     |
|    ep_rew_mean          | -2.23        |
| time/                   |              |
|    fps                  | 587          |
|    iterations           | 33           |
|    time_elapsed         | 114          |
|    total_timesteps      | 67584        |
| train/                  |              |
|    approx_kl            | 0.0034569954 |
|    clip_fraction        | 0.0322       |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.852       |
|    explained_variance   | -0.0287      |
|    learning_rate        | 0.001        |
|    loss                 | -0.0191      |
|    n_updates            | 320          |
|    policy_gradient_loss | -0.000945    |
|    value_loss           | 0.000222     |
------------------------------------------
---------------------------------

------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 3.31e+04     |
|    ep_rew_mean          | -2.23        |
| time/                   |              |
|    fps                  | 588          |
|    iterations           | 43           |
|    time_elapsed         | 149          |
|    total_timesteps      | 88064        |
| train/                  |              |
|    approx_kl            | 0.0059452783 |
|    clip_fraction        | 0.0788       |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.665       |
|    explained_variance   | 0.0168       |
|    learning_rate        | 0.001        |
|    loss                 | 0.0109       |
|    n_updates            | 420          |
|    policy_gradient_loss | -0.00563     |
|    value_loss           | 0.000105     |
------------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_m

...

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 3.31e+04    |
|    ep_rew_mean          | 0.154       |
| time/                   |             |
|    fps                  | 593         |
|    iterations           | 443         |
|    time_elapsed         | 1529        |
|    total_timesteps      | 907264      |
| train/                  |             |
|    approx_kl            | 0.012239219 |
|    clip_fraction        | 0.13        |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.573      |
|    explained_variance   | -0.0314     |
|    learning_rate        | 0.001       |
|    loss                 | -0.0109     |
|    n_updates            | 4420        |
|    policy_gradient_loss | -0.00444    |
|    value_loss           | 0.000215    |
-----------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 3.31e+

Market Return : 423.10%   |   Portfolio Return : -71.02%   |   
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 3.31e+04    |
|    ep_rew_mean          | 0.105       |
| time/                   |             |
|    fps                  | 593         |
|    iterations           | 453         |
|    time_elapsed         | 1563        |
|    total_timesteps      | 927744      |
| train/                  |             |
|    approx_kl            | 0.009783761 |
|    clip_fraction        | 0.117       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.505      |
|    explained_variance   | -0.0171     |
|    learning_rate        | 0.001       |
|    loss                 | -0.0214     |
|    n_updates            | 4520        |
|    policy_gradient_loss | -0.00589    |
|    value_loss           | 0.000151    |
-----------------------------------------
-----------------------------------------
| rollout/  

------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 3.31e+04     |
|    ep_rew_mean          | 0.105        |
| time/                   |              |
|    fps                  | 593          |
|    iterations           | 463          |
|    time_elapsed         | 1597         |
|    total_timesteps      | 948224       |
| train/                  |              |
|    approx_kl            | 0.0056185266 |
|    clip_fraction        | 0.0736       |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.397       |
|    explained_variance   | -0.0559      |
|    learning_rate        | 0.001        |
|    loss                 | 0.0116       |
|    n_updates            | 4620         |
|    policy_gradient_loss | -0.00133     |
|    value_loss           | 9.09e-05     |
------------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_m

------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 3.31e+04     |
|    ep_rew_mean          | 0.0929       |
| time/                   |              |
|    fps                  | 594          |
|    iterations           | 473          |
|    time_elapsed         | 1630         |
|    total_timesteps      | 968704       |
| train/                  |              |
|    approx_kl            | 0.0106566055 |
|    clip_fraction        | 0.115        |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.51        |
|    explained_variance   | 0.00673      |
|    learning_rate        | 0.001        |
|    loss                 | -0.0171      |
|    n_updates            | 4720         |
|    policy_gradient_loss | -0.00183     |
|    value_loss           | 0.0002       |
------------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_m

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 3.31e+04    |
|    ep_rew_mean          | 0.0929      |
| time/                   |             |
|    fps                  | 594         |
|    iterations           | 483         |
|    time_elapsed         | 1662        |
|    total_timesteps      | 989184      |
| train/                  |             |
|    approx_kl            | 0.007496462 |
|    clip_fraction        | 0.0847      |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.468      |
|    explained_variance   | 0.0221      |
|    learning_rate        | 0.001       |
|    loss                 | 0.00367     |
|    n_updates            | 4820        |
|    policy_gradient_loss | -0.00149    |
|    value_loss           | 0.000537    |
-----------------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 3.31

In [228]:
model.save("my_trading_agent")

In [229]:
# Create the environment directly
env2 = TradingEnv(
    name="BTCUSD",
    df=df,  # Your dataset with your custom features
    positions=[-0.8, 0, 0.8],  # -1 (=SHORT), 0(=OUT), +1 (=LONG)
    trading_fees=0.01/100,  # 0.01% per stock buy/sell (Binance fees)
    borrow_interest_rate=0.0003/100,  # 0.0003% per timestep (one timestep = 1h here)
)

In [230]:
# Run an episode until it ends :
done, truncated = False, False
obs, info = env2.reset()
while not done and not truncated:
    action, _ = model.predict(obs)
    obs, reward, done, truncated, info = env2.step(action)
    # ... (evaluate further)

Market Return : 423.10%   |   Portfolio Return : 240.58%   |   


In [231]:
# At the end of the episode you want to render
env2.unwrapped.save_for_render(dir="render_logs")